In [ ]:
!nvidia-smi -L
!pip install pyDOE

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
os.chdir('./gdrive/MyDrive/')
!pwd

In [ ]:
import numpy as np
import torch
from pyDOE import lhs
from torch import nn
from torch.nn import functional as F
from torch import optim
import time
from torch.utils.tensorboard import SummaryWriter 
import random

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # for multiGPUs.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
MODEL_NAME = 'test1lr0.1decay0'
N0 = 20
N1 = 2000
N_TEST = 1000
N_val = 500

V = 0.5
EPOCHS = 20000
LOG_EVERY_EPOCH = 1000
PLOT_EVERY_EPOCH = 2000

LAYERS =5
NEURONS_PER_LAYER = 100

GRAD_CLIP_VALUE = 5

ATTACK_STEPS = 8
eps_final = 0.05
attack_type = 'adv' # 'guassian' #or 'adv'

sampling_func = lhs
#sampling_func = equi_distant

In [ ]:
def equi_distant(dims, count):
    if dims == 1:
        return np.linspace(0, 1, count + 2)[1:-1].reshape(-1, 1)
    if dims == 2:
        count = int(np.ceil(np.sqrt(count)))
        line = np.linspace(0, 1, count + 2)[1:-1]
        x, y = np.meshgrid(line, line)
        return np.stack([x.reshape(-1), y.reshape(-1)], 1)

In [ ]:
writer = SummaryWriter(f'./PINN/KS_change_label/logs/{MODEL_NAME}/')
hyper_writer = SummaryWriter('./PINN/KS_change_label/SUMMARY/')

In [ ]:
class Model(nn.Module):
    def __init__(self, input_shape, output_shape, layers, neurons_per_layer):
        super().__init__()
        self.layers = layers
        self.fc1 = nn.Linear(input_shape, neurons_per_layer)
        for i in range(layers - 2):
            layer = nn.Linear(neurons_per_layer, neurons_per_layer)
            setattr(self, f'fc{i + 2}', layer)
        layer = nn.Linear(neurons_per_layer, output_shape)
        setattr(self, f'fc{layers}', layer)

        # Activaiton Function
        # self.activation = nn.LeakyReLU(0.1)
        self.activation = nn.Tanh()
        # self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        for i in range(self.layers - 1):
            layer = getattr(self, f'fc{i + 1}')
            x = layer(x)
            x = self.activation(x)
        layer = getattr(self, f'fc{self.layers}')
        x = layer(x)
        return x

In [ ]:
def derivative(u_, x_, order=1, ignore_dim=1):
    ones_ = torch.ones_like(u_)
    drv = torch.autograd.grad(u_, x_, create_graph=True, grad_outputs=ones_)[0]
    for i in range(1, order):
        ones_ = torch.ones_like(drv)
        ones_[:, ignore_dim] = 0
        drv = torch.autograd.grad(drv, x_, create_graph=True, grad_outputs=ones_)[0]
    return drv

In [ ]:
# (0, 20) x (0, 10)
MIN_X = 0
MAX_X = 20
MIN_T = 0
MAX_T = 1

x = torch.tensor(sampling_func(1, N0) * (MAX_X - MIN_X) + MIN_X).to(device)
zeros = torch.zeros_like(x)
ones = torch.ones_like(x)

D0 = torch.cat([x, zeros], 1)
Y0 = torch.sin(x)

D1 = torch.tensor(sampling_func(2, N1)).to(device)
D1[:, 0] = D1[:, 0] * (MAX_X - MIN_X) + MIN_X
D1[:, 1] = D1[:, 1] * (MAX_T - MIN_T) + MIN_T
P1 = torch.tensor([2 * np.pi, 0]).to(device)

D_tr = torch.tensor(sampling_func(2, N1)).to(device)
D_tr[:, 0] = D_tr[:, 0] * (MAX_X - MIN_X) + MIN_X
D_tr[:, 1] = D_tr[:, 1] * (MAX_T - MIN_T) + MIN_T
Y_tr = torch.sin(D_tr[:, 0:1] + D_tr[:, 1:2])
res_tr = torch.cos(D_tr[:, 0:1] + D_tr[:, 1:2]) * (1 + torch.sin(D_tr[:, 0:1] + D_tr[:, 1:2])) + (V - 1) * torch.sin(D_tr[:, 0:1] + D_tr[:, 1:2])

D_val= torch.tensor(lhs(2, N_val)).to(device)
D_val[:, 0] = D_val[:, 0] * (MAX_X - MIN_X) + MIN_X
D_val[:, 1] = D_val[:, 1] * (MAX_T - MIN_T) + MIN_T
Y_val = torch.sin(D_val[:, 0:1] + D_val[:, 1:2])
res_val = torch.cos(D_val[:, 0:1] + D_val[:, 1:2]) * (
    1 + torch.sin(D_val[:, 0:1] + D_val[:, 1:2])
) + (V - 1) * torch.sin(D_val[:, 0:1] + D_val[:, 1:2])

D_tst = torch.tensor(lhs(2, N_TEST)).to(device)
D_tst[:, 0] = D_tst[:, 0] * (MAX_X - MIN_X) + MIN_X
D_tst[:, 1] = D_tst[:, 1] * (MAX_T - MIN_T) + MIN_T
Y_tst = torch.sin(D_tst[:, 0:1] + D_tst[:, 1:2])
res_tst = torch.cos(D_tst[:, 0:1] + D_tst[:, 1:2]) * (
    1 + torch.sin(D_tst[:, 0:1] + D_tst[:, 1:2])
) + (V - 1) * torch.sin(D_tst[:, 0:1] + D_tst[:, 1:2])

In [ ]:
model = Model(2, 1, LAYERS, NEURONS_PER_LAYER).to(device).double()
#optimizer = optim.Adam(model.parameters(), lr = 0.01)
optimizer = optim.Adamax(model.parameters(), lr=0.01, weight_decay=0)
#optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, 1000, 0.5)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 50000, eta_min=0, last_epoch=-1, verbose=False)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)

In [ ]:
def mse_output_target(x, y):
    u = model(x)
    return F.mse_loss(u, y)


def mse_periodic(x, y):
    u = model(x)
    x2 = x + y
    u2 = model(x2)
    return F.mse_loss(u, u2)


def mse_residual(x, y):

    x = x.requires_grad_(True)
    u = model(x)
    u_d = derivative(u, x)
    u_dd = derivative(u, x, 2)
    u_dddd = derivative(u, x, 4)

    u_t = u_d[:, 1:2]
    u_x = u_d[:, 0:1]
    
    u_xx = u_dd[:, 0:1]
    u_xxxx = u_dddd[:, 0:1]

    return F.mse_loss(u_t + u * u_x + u_xx + V * u_xxxx, y)

In [ ]:
#@title Plot
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

def plot_model(title=None, epoch=None):
    fig = plt.figure(figsize=plt.figaspect(0.5) * 1.5)
    ax = fig.add_subplot(1, 2, 1, projection="3d")

    X = np.linspace(MIN_X, MAX_X, 300)
    T = np.linspace(MIN_T, MAX_T, 300)
    X, T = np.meshgrid(X, T)
    u = torch.tensor(np.stack([X.reshape(-1), T.reshape(-1)], 1)).to(device)
    Z = model(u).cpu().detach().numpy()
    U = np.sin(X + T)
    Z = Z.reshape(X.shape)
    ax.plot_surface (X, T, Z
                    , rstride=1 # default value is one
                    , cstride=1 # default value is one
                    , cmap='winter'
                    , edgecolor='none'
                    )
    ax.scatter3D(D_tr[:, 0].cpu().detach().numpy(),
                 D_tr[:, 1].cpu().detach().numpy(),
                 Y_tr[:, 0].cpu().detach().numpy(),
                 marker='x')
    ax.scatter3D(D0[:, 0].cpu().detach().numpy(),
                 D0[:, 1].cpu().detach().numpy(),
                 Y0[:, 0].cpu().detach().numpy(),
                 marker='x')
    
    ax.set_xlabel('X')
    ax.set_ylabel('T')
    ax.set_zlabel('Z')
    ax.set_title('Prediction');
    ###############################
    ax = fig.add_subplot(1, 2, 2, projection="3d")
    ax.plot_surface (X, T, U
                    , rstride=1 # default value is one
                    , cstride=1 # default value is one
                    , cmap='winter'
                    , edgecolor='none'
                    )
    ax.set_xlabel('X')
    ax.set_ylabel('T')
    ax.set_zlabel('Z')
    ax.set_title('Actual');
    if title is not None:
        fig.suptitle(f'{title}')
    if epoch is not None:
        writer.add_figure(
            'Function',
            fig,
            epoch,
            close=False
        )
    plt.show()


    fig = plt.figure(figsize=plt.figaspect(0.5) * 1.5)
    ax = fig.add_subplot(1, 2, 1)

    ax.contour  (X, T, Z
                , cmap = plt.cm.autumn
                )
    ax.scatter(D_tr[:, 0].cpu().detach().numpy(),
               D_tr[:, 1].cpu().detach().numpy(),
               marker='x')
    
    ax.set_xlabel('X')
    ax.set_ylabel('T')
    ax.set_title('Prediction')

    ##################################

    ax = fig.add_subplot(1, 2, 2)
    ax.contour  (X, T, U
                , cmap = plt.cm.autumn
                )
    ax.set_xlabel('X')
    ax.set_ylabel('T')
    ax.set_title('Actual')
    if epoch is not None:
        writer.add_figure(
            'Contour',
            fig,
            epoch,
            close=False
        )

    plt.show()

In [ ]:
plot_model('init')

In [ ]:
#%load_ext tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
!ps aux | grep tensorboard 

In [ ]:
%tensorboard --logdir ./PINN/KS_change_label/

In [ ]:
%cd '/content/'
KSchangelabel_path = '/content/gdrive/MyDrive/KSchangelabel.pth' 
best_path_adv = '/content/gdrive/MyDrive/best_adv.pth' 
resume = 0

start_epoch = 0
min_error_adv = 999999
if resume:
    checkpoint = torch.load(KSchangelabel_path, map_location=torch.device('cpu')) #torch.load(KSchangelabel_path)
    model.load_state_dict(checkpoint['net'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    start_epoch = checkpoint['epoch']+1
    min_error_adv = checkpoint['best_error']

In [ ]:
#ATTACK_STEPS = 8

def attack(x, target, loss_function, steps, eps_1 = 20*eps_final, eps_2 = 1*eps_final):

    eps = torch.ones_like(x)
    eps[:, 0] = eps[:, 0] * eps_1
    eps[:, 1] = eps[:, 1] * eps_2
    delta = (eps/steps)*1.5

    noise = torch.zeros_like(x).requires_grad_(True)
    for i in range(steps):
      loss = loss_function(x + noise, target)
      loss.backward()
      with torch.no_grad():
        grad = noise.grad
        #grad /= torch.norm(grad, p=2, dim=1, keepdim=True)  # p=float('inf')
        grad = grad.sign()


        noise += grad * delta
        #norm2 = torch.norm(noise, p=2, dim=1, keepdim=True)   # p=float('inf')
        #norm2 = torch.maximum(norm2, torch.ones_like(norm2) * eps)
        #noise *= eps / norm2

        noise[:, 0][noise[:, 0]>eps_1] = eps_1
        noise[:, 0][noise[:, 0]<-eps_1] = -eps_1

        noise[:, 1][noise[:, 1]>eps_2] = eps_2
        noise[:, 1][noise[:, 1]<-eps_2] = -eps_2
        #noise = torch.clamp(noise, min=-eps, max=eps)
        
        noise.grad.zero_()

    return noise.detach()


def attack_change_label(x, loss_function, steps, eps_1 = 20*eps_final, eps_2 = 1*eps_final, type_points='D0'):

    eps = torch.ones_like(x)
    eps[:, 0] = eps[:, 0] * eps_1
    eps[:, 1] = eps[:, 1] * eps_2
    delta = (eps/steps)*1.5

    noise = torch.zeros_like(x).requires_grad_(True)
    for i in range(steps):

      if type_points=='D0':
        target = torch.sin((x + noise)[:, 0].unsqueeze(1))
      elif type_points=='D1':
        target = torch.tensor([2 * np.pi, 0]).to(device)
      elif type_points=='D_tr':
        target = torch.cos((x + noise)[:, 0:1] + (x + noise)[:, 1:2]) * (1 + torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])) + (V - 1) * torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])

      loss = loss_function(x + noise, target)
      loss.backward()
      with torch.no_grad():
        grad = noise.grad
        grad = grad.sign()


        noise += grad * delta

        noise[:, 0][noise[:, 0]>eps_1] = eps_1
        noise[:, 0][noise[:, 0]<-eps_1] = -eps_1

        noise[:, 0][noise[:, 0]>(MAX_X-x[:, 0])] = MAX_X-x[:, 0][noise[:, 0]>(MAX_X-x[:, 0])]
        noise[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] = MIN_X-x[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] 

        noise[:, 1][noise[:, 1]>eps_2] = eps_2
        noise[:, 1][noise[:, 1]<-eps_2] = -eps_2

        noise[:, 1][noise[:, 1]>(MAX_T-x[:, 1])] = MAX_T-x[:, 1][noise[:, 1]>(MAX_T-x[:, 1])]
        noise[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] = MIN_T-x[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] 
        
        noise.grad.zero_()
    
    if type_points=='D0':
      target = torch.sin((x + noise)[:, 0].unsqueeze(1))
    elif type_points=='D1':
      target = torch.tensor([2 * np.pi, 0]).to(device)
    elif type_points=='D_tr':
      target = torch.cos((x + noise)[:, 0:1] + (x + noise)[:, 1:2]) * (1 + torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])) + (V - 1) * torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])


    return noise.detach(), target.detach()


def attack_change_label_gussaian(x, loss_function, steps, eps_1 = 20*eps_final, eps_2 = 1*eps_final, type_points='D0'):

    eps = torch.zeros_like(x)
    eps[:, 0] = eps_1
    eps[:, 1] = eps_2

    noise = torch.randn(x.size(), device=device).requires_grad_(True) * eps

    noise[:, 0][noise[:, 0]>(MAX_X-x[:, 0])] = MAX_X-x[:, 0][noise[:, 0]>(MAX_X-x[:, 0])]
    noise[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] = MIN_X-x[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] 

    noise[:, 1][noise[:, 1]>(MAX_T-x[:, 1])] = MAX_T-x[:, 1][noise[:, 1]>(MAX_T-x[:, 1])]
    noise[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] = MIN_T-x[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] 

    if type_points=='D0':
      target = torch.sin((x + noise)[:, 0].unsqueeze(1))
    elif type_points=='D1':
      target = torch.tensor([2 * np.pi, 0]).to(device)
    elif type_points=='D_tr':
      target = torch.cos((x + noise)[:, 0:1] + (x + noise)[:, 1:2]) * (1 + torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])) + (V - 1) * torch.sin((x + noise)[:, 0:1] + (x + noise)[:, 1:2])

    return noise, target.detach()

def attack_without_change_label_gussaian(x, loss_function, steps, eps_1 = 20*eps_final, eps_2 = 1*eps_final, type_points='D0'):

    eps = torch.zeros_like(x)
    eps[:, 0] = eps_1
    eps[:, 1] = eps_2

    noise = torch.randn(x.size(), device=device).requires_grad_(True) * eps

    noise[:, 0][noise[:, 0]>(MAX_X-x[:, 0])] = MAX_X-x[:, 0][noise[:, 0]>(MAX_X-x[:, 0])]
    noise[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] = MIN_X-x[:, 0][noise[:, 0]<(MIN_X-x[:, 0])] 

    noise[:, 1][noise[:, 1]>(MAX_T-x[:, 1])] = MAX_T-x[:, 1][noise[:, 1]>(MAX_T-x[:, 1])]
    noise[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] = MIN_T-x[:, 1][noise[:, 1]<(MIN_T-x[:, 1])] 

    if type_points=='D0':
      target = torch.sin((x)[:, 0].unsqueeze(1))
    elif type_points=='D1':
      target = torch.tensor([2 * np.pi, 0]).to(device)
    elif type_points=='D_tr':
      target = torch.cos((x)[:, 0:1] + (x)[:, 1:2]) * (1 + torch.sin((x)[:, 0:1] + (x)[:, 1:2])) + (V - 1) * torch.sin((x)[:, 0:1] + (x)[:, 1:2])

    return noise, target.detach()

In [ ]:
if attack_type == 'attack_change_label_gussaian':
  attack_type = attack_change_label_gussaian

if attack_type == 'attack_without_change_label_gussaian':
  attack_type = attack_without_change_label_gussaian

elif attack_type == 'adv':
  attack_type = attack_change_label

In [ ]:
all_train_loss = []
all_val_loss = []
all_epochs = []

print('Start epoch: ', start_epoch)
epoch_start = time.time()
for epoch in range(start_epoch, EPOCHS):
    model.train()
    optimizer.zero_grad()

    if epoch <= int(EPOCHS/2):
        eps_1=eps_final*epoch/int(EPOCHS/2)
        eps_2=eps_final*epoch/int(EPOCHS/2)
    else:
        eps_1=eps_final
        eps_2=eps_final

    #noise_ud0 = attack(D0, Y0, mse_output_target, ATTACK_STEPS, 20*eps_1, eps_2 = 0)
    noise_ud0, noise_Y0 = attack_type(D0, mse_output_target, ATTACK_STEPS, 20*eps_1, eps_2 = 0, type_points='D0')
    #noise_ud1 = attack(D1, P1, mse_periodic, ATTACK_STEPS, 20*eps_1, eps_2)
    noise_ud1, noise_Y1 = attack_type(D1, mse_periodic, ATTACK_STEPS, 20*eps_1, eps_2, type_points='D1')
    ud0 = model(D0 + noise_ud0)  # or u0t = model(torch.clamp(D0 + noise_u0t))
    # ud1 = model(D1 + noise_ud1)
    #loss_0 = F.mse_loss(ud0, Y0)
    loss_0 = F.mse_loss(ud0, noise_Y0)
    #loss_1 =  mse_periodic(D1 + noise_ud1, P1) ##check
    loss_1 =  mse_periodic(D1 + noise_ud1, noise_Y1)


    #noise_u = attack(D_tr, res_tr, mse_residual, ATTACK_STEPS, 20*eps_1, eps_2)
    noise_u, noise_res_tr = attack_type(D_tr, mse_residual, ATTACK_STEPS, 20*eps_1, eps_2, type_points='D_tr')
    adv_input = D_tr + noise_u ##check (add model)
    #loss_2 = mse_residual(adv_input, res_tr)
    loss_2 = mse_residual(adv_input, noise_res_tr)


    #loss_check = [F.mse_loss(model(D0), Y0).mean().item(), mse_periodic(D1, P1).mean().item(), mse_residual(D_tr, res_tr).mean().item()]
    #loss_check2 = [loss_0.mean().item(), loss_1.mean().item(), loss_2.mean().item()]
    #print(loss_check, loss_check2)

    loss = loss_0 + loss_1 + loss_2
    writer.add_scalars('loss', {
        'Total': loss.item(),
        'Boundary': loss_0.item(),
        'Periodic': loss_1.item(),
        'Collocation': loss_2.item()
    }, epoch)

    loss.backward()
    nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP_VALUE)
    optimizer.step()
    scheduler.step()

    if epoch % LOG_EVERY_EPOCH == 0:
        print('Epoch', epoch)
        model.eval()
        u = model(D_tr)
        function_loss = F.mse_loss(u, Y_tr).item()
        print(f'Training loss {loss.item()}, function loss {function_loss}')

        noise_val, Y_new = attack_type(D_val, mse_residual, ATTACK_STEPS, 20*eps_1, eps_2, type_points='D_tr')
        u_val = model(D_val)
        val_function_loss = F.mse_loss(u_val, Y_val).item()
        u_val_adv = model(D_val + noise_val)
        adv_val_function_loss = F.mse_loss(u_val_adv, Y_new).item()
        print(f'val function loss: Clean {val_function_loss}, '
              f'Adv {adv_val_function_loss}')
        print('time', time.time() - epoch_start)
        epoch_start = time.time()
        writer.add_scalars('Function Loss', {
            'Train': function_loss,
            'val': val_function_loss,
            'val Adv': adv_val_function_loss,
        }, epoch)
        writer.flush()

        all_train_loss.append(function_loss)
        all_val_loss.append(val_function_loss)
        all_epochs.append(epoch)
        
        if val_function_loss<min_error_adv:
            min_error_adv = val_function_loss

            state = {'net': model.state_dict(),
              'epoch': epoch,
              'optimizer': optimizer.state_dict(),
              'scheduler': scheduler.state_dict(),
              'best_error' : min_error_adv
              }

          
            torch.save(state, best_path_adv)
            print('checkpoint epoch: ', epoch)
        
        
        
    if epoch % PLOT_EVERY_EPOCH == 0:
        plot_model(f'Epoch {epoch}', epoch)
    
    if epoch % 50 == 0:
        state = {'net': model.state_dict(),
              'epoch': epoch,
              'optimizer': optimizer.state_dict(),
              'scheduler': scheduler.state_dict(),
              'best_error' : min_error_adv
              }

          
        torch.save(state, KSchangelabel_path)
        print('checkpoint epoch: ', epoch)

In [ ]:
%cd /content/
torch.save([all_train_loss,
all_val_loss,
all_epochs], 'KS_PIAT.pth')

In [ ]:
#Test

checkpoint = torch.load(best_path_adv)
model.load_state_dict(checkpoint['net'])

model.eval()
u = model(D_tr)
function_loss = F.mse_loss(u, Y_tr).item()
print(f'Training loss {loss.item()}, function loss {function_loss}')

noise_test = attack(D_tst, res_tst, mse_residual, ATTACK_STEPS, 20*eps_1, eps_2)
u_test = model(D_tst)
test_function_loss = F.mse_loss(u_test, Y_tst).item()
u_test_adv = model(D_tst + noise_test)
adv_test_function_loss = F.mse_loss(u_test_adv, Y_tst).item()
print(f'Test function loss: Clean {test_function_loss}, '
      f'Adv {adv_test_function_loss}')
print('time', time.time() - epoch_start)
epoch_start = time.time()
writer.add_scalars('Function Loss', {
    'Train': function_loss,
    'Test': test_function_loss,
    'Test Adv': adv_test_function_loss,
}, epoch)
writer.flush()


In [ ]:
def evaluate(D, Y):
    u = model(D)
    function_loss = F.mse_loss(u, Y).item()
    return function_loss

In [ ]:
'''
model.eval()
train_function_loss = evaluate(D_tr, Y_tr)
print(f'Training loss {loss.item()}, function loss {train_function_loss}')

noise_test = attack(D_tst, res_tst, mse_residual, 8, 0.05, 0.01, 20*eps_1, eps_2)

test_function_loss = evaluate(D_tst, Y_tst)

adv_test_function_loss = evaluate(D_tst + noise_test, Y_tst)
print(f'Test function loss: Clean {test_function_loss}, '
      f'Adv {adv_test_function_loss}')
'''

In [ ]:
'''
hyper_writer.add_hparams(
    {
        'N0': N0,
        'N1': N1,
        'N_TEST': N_TEST,
        'EPOCHS': EPOCHS,
        'LAYERS': LAYERS,
        'NEURONS_PER_LAYER': NEURONS_PER_LAYER,
        'ATTACK_STEPS': ATTACK_STEPS,
        'ATTACK_DELTA': ATTACK_DELTA,
        'ATTACK_EPS': ATTACK_EPS,
        'SAMPLING_FUNCTION': sampling_func.__name__,
        'ACTIVATION_FUNCTION': str(model.activation),
    #    'ATTACK_TYPE': str(ATTACK_TYPE),
     
    },
    {
        'training_function_loss': train_function_loss,
        'test_clean_function_loss': test_function_loss,
        'test_adv_function_loss': adv_test_function_loss,
    }
)
'''

In [ ]:
writer.close()
hyper_writer.close()